# Accès aux soins de santé - Rapport

## Proportion de la population vivant dans un rayon de 5 km autour d'un établissement de soins de santé (environ 60 minutes à pied)

In [ ]:
#%% Clean environment & global settings
options(scipen=999)
# Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
# Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")

In [ ]:
#%% File paths

ROOT_PATH <- '~/workspace'
PROJECT_PATH <- file.path(ROOT_PATH, "pipelines/snt_healthcare_access")
CONFIG_PATH <- file.path(ROOT_PATH, 'configuration')
CODE_PATH <- file.path(ROOT_PATH, 'code')
DATA_PATH <- file.path(ROOT_PATH, 'data')
DHIS2_DATA_PATH <- file.path(DATA_PATH, 'dhis2', 'formatted')
OUTPUT_DATA_PATH <- file.path(DATA_PATH, 'healthcare_access')
OUTPUT_PLOTS_PATH <- file.path(PROJECT_PATH, 'reporting', 'outputs')

In [ ]:
#%% Load utils, config and libraries
source(file.path(CODE_PATH, "snt_utils.r"))

# Required packages
required_packages <- c(
  "jsonlite",
  "dplyr",
  "data.table",
  "ggplot2",
  "arrow",
  "glue",
  # "geojsonio",
  "sf",
  "terra",
  "RColorBrewer",
  "httr",
  "reticulate",
  "arrow"
)

# Execute function
install_and_load(required_packages)

Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")
reticulate::py_config()$python
openhexa <- import("openhexa.sdk")

# Load SNT config
CONFIG_FILE_NAME <- "SNT_config.json"
config_json <- tryCatch({ fromJSON(file.path(CONFIG_PATH, CONFIG_FILE_NAME)) },
                        error = function(e) {
                          msg <- paste0("Error while loading configuration", conditionMessage(e))  
                          cat(msg)   
                          stop(msg) 
                        })

msg <- paste0("SNT configuration loaded from  : ", file.path(CONFIG_PATH, CONFIG_FILE_NAME)) 
log_msg(msg)

In [ ]:
#%% Config variables

COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE
print(paste("Country code: ", COUNTRY_CODE))

dhis2_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED

In [ ]:
#%% Global variables

admin_col <- "ADM2_ID"
admin_level <- "ADM2"
# data_source <- "WPOP"
reference_year <- 2020
# country_epsg_degrees <- 4326 # for plotting
# country_epsg_meters <- 32630 # for creating the buffer areas

# column names
latitude_col <- "LATITUDE"
longitude_col <- "LONGITUDE"
coordinate_cols <- c(longitude_col, latitude_col) # longitude (x) first, latitude (y) second

## Spatial data

In [ ]:
# spatial data filename
filename_spatial_units_data <- paste(COUNTRY_CODE, 'shapes.geojson', sep = '_')

# load as vector data
spatial_units_data <- tryCatch({ get_latest_dataset_file_in_memory(dhis2_dataset, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 Shapes data for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })

In [ ]:
dt_filename_stem <- glue("{COUNTRY_CODE}_population_covered_health")
dt <- read_parquet(file.path(OUTPUT_DATA_PATH, glue("{dt_filename_stem}.parquet")))
setDT(dt)

In [ ]:
# compute the complement: percentage of population not within reach of a FOSA
dt[, PCT_UNCOVERED := 100 - PCT_HEALTH_ACCESS]

In [ ]:
# make categories
dt <- dt[, CAT_UNCOVERED := cut(PCT_UNCOVERED,
                    breaks = seq(0, 100, by = 20),
                    include.lowest = TRUE,
                    right = TRUE,
                    labels = paste0(seq(0, 80, 20), "-", seq(20, 100, 20)))]

In [ ]:
plot_data <- merge(spatial_units_data, dt, by = c("ADM1_ID", "ADM1_NAME", "ADM2_ID", "ADM2_NAME"), all.x = TRUE)

In [ ]:
pop_covered_plot <- ggplot(plot_data) +
  geom_sf(aes(fill = PCT_HEALTH_ACCESS)) +
  scale_fill_viridis_c(option = "plasma") +  # for continuous data
  # scale_fill_distiller(palette = "RdYlGn", direction = 1)
  theme_minimal() +
  ggtitle(glue("Population within healthcare coverage in {COUNTRY_CODE} (%)"))

In [ ]:
print(pop_covered_plot)

In [ ]:
# Save as .png file

pop_covered_plot_filename <- glue("{COUNTRY_CODE}_{admin_level}_{reference_year}_pop_health_covered_plot.png")
ggsave(file.path(OUTPUT_PLOTS_PATH, pop_covered_plot_filename), plot = pop_covered_plot)

In [ ]:
uncovered_cat_palette <- colorRampPalette(c("white", "#59141B"))(5)

cat_uncovered_plot <- ggplot(plot_data) +
  geom_sf(aes(fill = CAT_UNCOVERED)) +
  scale_fill_manual(values = uncovered_cat_palette, drop = FALSE) +
  theme_minimal() +
  labs(title = glue("Population outside healthcare coverage in {COUNTRY_CODE} (20% intervals)"),
       fill = "Category")

print(cat_uncovered_plot)

cat_uncovered_plot_filename <- glue("{COUNTRY_CODE}_{admin_level}_{reference_year}_cat_pop_health_uncovered_plot.png")

ggsave(file.path(OUTPUT_PLOTS_PATH, cat_uncovered_plot_filename), plot = cat_uncovered_plot)

## Carte des structures sanitaires géolocalisées

Cette carte montre les structures sanitaires disposant de coordonnées GPS plausibles, localisées à l'intérieur des frontières du pays.

In [ ]:
# Load FOSA data from DHIS2 pyramid and map valid geolocated facilities
pyramid_file <- paste0(COUNTRY_CODE, "_pyramid.parquet")
pyramid_data <- tryCatch(
  get_latest_dataset_file_in_memory(dhis2_dataset, pyramid_file),
  error = function(e) {
    msg <- paste("Error while loading pyramid file:", pyramid_file, conditionMessage(e))
    cat(msg)
    stop(msg)
  }
)

fosa_level <- config_json$SNT_CONFIG$ANALYTICS_ORG_UNITS_LEVEL
if ("LEVEL" %in% names(pyramid_data)) {
  fosa_data <- pyramid_data[pyramid_data$LEVEL == fosa_level, ]
} else {
  fosa_data <- pyramid_data
}

# Identify coordinate columns (case-insensitive)
lat_col <- grep("^LATITUDE$", names(fosa_data), ignore.case = TRUE, value = TRUE)[1]
lon_col <- grep("^LONGITUDE$", names(fosa_data), ignore.case = TRUE, value = TRUE)[1]

if (is.na(lat_col) || is.na(lon_col)) {
  stop("Could not find LATITUDE/LONGITUDE columns in pyramid data.")
}

fosa_points <- as.data.table(fosa_data)
fosa_points[, LAT_NUM := suppressWarnings(as.numeric(get(lat_col)))]
fosa_points[, LON_NUM := suppressWarnings(as.numeric(get(lon_col)))]

# Keep plausible coordinates only
fosa_points <- fosa_points[
  !is.na(LAT_NUM) & !is.na(LON_NUM) &
    LAT_NUM >= -90 & LAT_NUM <= 90 &
    LON_NUM >= -180 & LON_NUM <= 180
]

# Convert to sf and keep only facilities within country boundaries
fosa_sf <- st_as_sf(fosa_points, coords = c("LON_NUM", "LAT_NUM"), crs = 4326)
country_polygon <- st_union(spatial_units_data)
fosa_sf_in_country <- st_filter(fosa_sf, country_polygon, .predicate = st_within)

n_total <- nrow(fosa_data)
n_plausible <- nrow(fosa_sf)
n_in_country <- nrow(fosa_sf_in_country)
n_outside <- n_plausible - n_in_country
pct_outside_total <- ifelse(n_total > 0, round(100 * n_outside / n_total, 1), NA_real_)
pct_outside_plausible <- ifelse(n_plausible > 0, round(100 * n_outside / n_plausible, 1), NA_real_)

msg <- paste0(
  "Facilities with plausible GPS in-country: ", n_in_country, " / ", n_total,
  " | outside boundaries: ", n_outside,
  " (", pct_outside_total, "% of total; ", pct_outside_plausible, "% of plausible GPS points)"
)
log_msg(msg)

fosa_geo_qc <- data.frame(
  METRIC = c(
    "Total facilities",
    "Facilities with plausible GPS",
    "Facilities with plausible GPS in-country",
    "Facilities with plausible GPS outside boundaries"
  ),
  COUNT = c(n_total, n_plausible, n_in_country, n_outside),
  PCT_OF_TOTAL = c(
    100,
    ifelse(n_total > 0, round(100 * n_plausible / n_total, 1), NA_real_),
    ifelse(n_total > 0, round(100 * n_in_country / n_total, 1), NA_real_),
    pct_outside_total
  )
)

print(fosa_geo_qc)

fosa_map_plot <- ggplot() +
  geom_sf(data = spatial_units_data, fill = "grey95", color = "grey60", linewidth = 0.2) +
  geom_sf(data = fosa_sf_in_country, color = "#08519c", alpha = 0.8, size = 0.5) +
  theme_minimal() +
  labs(
    title = glue("Structures sanitaires géolocalisées - {COUNTRY_CODE}"),
    subtitle = glue("Points GPS plausibles situés dans les frontières nationales (n={nrow(fosa_sf_in_country)})"),
    x = NULL,
    y = NULL
  )

print(fosa_map_plot)

fosa_map_plot_filename <- glue("{COUNTRY_CODE}_fosa_geolocated_map.png")
ggsave(file.path(OUTPUT_PLOTS_PATH, fosa_map_plot_filename), plot = fosa_map_plot, width = 10, height = 7)